# Connecting with the Credo AI Governance App

Connecting with the Credo AI Governance App is straightforward using Lens. This notebook is a comprehensive overview of how you you can interact with the Governance App.

## Setup

**Find the code**

This notebook can be found on [github](https://github.com/credo-ai/credoai_lens/blob/develop/docs/notebooks/governance_integration.ipynb).

**Config File**

First, ensure you have a config file somewhere on your computer named ".credoconfig". The default location where Lens will look for it in your home directory (`~/.credoconfig`). The structure of the config should look like this

```
TENANT=<tenant name> # Example: credoai
CREDO_URL=<your credo url> 
API_KEY=<your api key> # Example: JSMmd26...
```

This config gives Lens API access to the Governance App. A config file can be downloaded from the web app. MySettings -> Token

**Governance App Setup**
Connecting Lens to the Governance App requires that you have already defined a Use-Case and registered a model to that use-case. That model serves as the endpoint for Lens.

**Data + Model Preparation (before Lens)**

Some quick setup. This script reflects all of your datascience work before assessment and integration with Credo AI.

Here we have a gradient boosted classifier trained on the UCI Credit Card Default Dataset.

In [1]:
# model and df are defined by this script
%run training_script.py

### Imports

In [2]:
# Base Lens imports
import credoai.lens as cl
# set default format for image displays. Change to 'png' if 'svg' is failing
%config InlineBackend.figure_formats = ['svg']

## Integration Quickstart

Building off of the [quickstart notebook](https://credoai-lens.readthedocs.io/en/latest/notebooks/quickstart.html), let's look at how you can export to a Use Case on the Governance App. We only have to add two new lines.

**Note** The below will fail unless you change "assessment-spec-destination" to a assessment-spec-destination related to an assessment spec actually defined in your Governance app! 

The assessment-spec will have the following form:
`use_cases/{use_case_id}/models/{model_id}/assessment_spec`

In [ ]:
credo_model = cl.CredoModel(name='credit_default_classifier',
                            model=model)

credo_data = cl.CredoData(name='UCI-credit-default',
                          data=df,
                          sensitive_feature_keys=['SEX'],
                          label_key='target'
                          )

# specify the metrics that will be used by the Fairness and Performance assessment
# run lens
lens = cl.Lens(model=credo_model,
               data=credo_data,
               governance='assessment-spec-destination' # <- NEW LINE!
               )

lens.run_assessments().create_report().export() # <- Added export

Those two lines did a number of things under the hood.

* First a [CredoGovernance Artifact](#Credo-Governance-Artifact) was created with the credo-url provided.
* Next the datasets were registered:
    * The dataset was registered to the Governanc App
    * The dataset was registered as validation data for the associated model and use-case.
* Next an assessment plan was downloaded and updated:
    * An assessment plan was downloaded for the model (if it exists).
    * The assessment plan was updated with the assessment_plan specified in code here.
* Finally, when `export` was called the assessment was exported to the model under the use-case.

## In-Depth Overview

### Credo Governance Artifact

Lens connects to the Governance App via a `CredoGovernance` object. All you need to do is provide a `spec-destination`. The `spec-destination` can be one of two things:
* A url for the spec in the AI Governance app (which can be found in the Governance App, or in an email requesting a model assessment))
* A file path to the spec after it was downloaded from the AI Governance App (an example assessment_plan.json is included in the repo for this package and will be referenced below).

**The CredoGovernance object has a few functionalities.**

* Retrieving an alignment spec from the Credo AI Governance App. The alignment spec is either downloaded straight from the Governance App based on a credo_url, or supplied as a json file for air gap deployment. 
    * This alignment spec includes an assessment plan, which can be supplemented (or overwritten) with an assessment plan you define in code and pass to Lens.
* Registering artifacts (e.g., datasets) to the Governance App.
* Encoding the IDs of governance objects so Lens can easily export metrics and reports.

Let's see how this works in practice.

In [3]:
# Here we reference a local assessment plan, rather than providing a url!
# Normally you would only do this in an air-gap environment
credo_gov = cl.CredoGovernance(spec_destination = 'assessment_plan.json')

# one aspect of the assessment spec is an assessment plan...
credo_gov.get_assessment_plan()

defaultdict(dict, {'Performance': {'metrics': ['precision_score']}})

The above assessment plan was defined by the assessment spec. It's a primary way in which the Governance App communicates with Lens and defines an assessment strategy.

In [4]:
# define the other artifacts as normal
credo_model = cl.CredoModel(name='credit_default_classifier',
                            model=model)

credo_data = cl.CredoData(name='UCI-credit-default',
                          data=df,
                          sensitive_feature_keys=['SEX'],
                          label_key='target'
                         )

# specify the metrics that will be used by the Fairness assessment. 
# This plan will supplement the plan that is defined in the assessment spec!
alignment_plan = {
    'Fairness': {'metrics': ['precision_score', 'recall_score']}
}

### Interacting with Credo AI's Governance App from Lens

In addition to the model and data, we can pass the CredoGovernance object. This is how Lens knows which Use Case, model and/or data to interacts with. If you are running assessments on multiple models, you'll create a separate `CredoGovernance` object (and separate Lens instances) for each.

You can also pass the `credo-url` and nothing else. In this case a CredoGovernance object will be created.

In [5]:
from credoai.assessment import FairnessAssessment, PerformanceAssessment
lens = cl.Lens(model=credo_model,
               data=credo_data,
               governance=credo_gov,
               assessments = [PerformanceAssessment, FairnessAssessment],
               assessment_plan=alignment_plan)

# After running, you just have to export and the metrics 
# will be sent to the Governance App
results = lens.run_assessments().get_results()

INFO:absl:Automatically Selected Assessments for Model without data

Assessment will not be run
Assessment will not be run
INFO:absl:Automatically Selected Assessments for model: credit_default_classifier and validation dataset: UCI-credit-default
--
INFO:absl:Selected assessments...
--Performance
--Fairness
INFO:absl:Initializing assessments for validation dataset: UCI-credit-default
INFO:absl:Running assessment-Performance
INFO:absl:Running assessment-Fairness


#### Reports

Reporting is a critical feature of Lens. Your assessments must be packaged in an easily digestible way to support downstream governance. As such, Lens supports robust reporting functionality.

Lens creates a jupyter notebook report that collates all of your assessment results into one place. It then converts that notebook to HTML and sends it to Credo AI's Governance App, along with your metrics when `export` is called. The report will be associated with that use-case, as well as a model (either the model you supplied to CredoGovernance, or the model [registered](#Registering-a-model-while-assessing-it) by Lens).

In [ ]:
lens.create_report()

## Air Gap Environment

If you cannot interact with Credo AI's Governance App via the internet, this section is for you!

Instead of Lens automating the communication between your assessment environment and governance, you will instead have to download an assessment_spec, provided it to Lens, and then upload the results assessments. The demo has actually already been doing this with an example spec included in the github repo!

### Getting the assessment spec
The asset that you have to get _from_ the governance app is the assessment spec. This can be found under your use case's "Risk Assessment" tab inside "Align". Once you download the assessments spec, you can read it with a CredoGovernance object by changing one argument.


In [ ]:
gov = cl.CredoGovernance(spec_destination='{path to assessment spec}')

Following that, you can pass the governance object to Lens as normal.

### Exporting Results

Lens can export assessment results to file as a json object. This is generally useful if you want to store your results locally, but particularly useful if you are in an air gapped environment where you cannot access Credo AI's Governance App directly from your development environment.

Doing this only requires a one line change in our current flow. We change the `export` argument from `credoai` to a local path:


In [ ]:
lens.run_assessments().create_report().export('{path where assessments should be exported}')

Running the above will create a folder that you specified, if it doesn't exist. It will then create a json file with the assessment results and the report, which can be uploaded to Credo AI's Governance App. This json file can be uploaded by going to the assessments of your **Use-Case** and pressing the purple "upload assessment" button.